### Load dataset

In [ ]:
import pandas as pd
from enum import StrEnum, auto
RANDOM_STATE = 42

# NOTE: StrEnum requires Python 3.11+
#       Can refactor to CONSTANTS instead
class FeatureVariant(StrEnum):
    LITERATURE = auto()
    RESEARCH = auto()
    STATISTICAL = auto()
    AUTOMATED = auto()

# Can be replaced with desired variant for different feature sets
GENE_FILE_VARIANT = FeatureVariant.LITERATURE

FILE_PATH = f"../Data/patient_genes_{GENE_FILE_VARIANT}.csv"
print(FILE_PATH)

variant = 'logistic_regression'
df = pd.read_csv(FILE_PATH)

df.info()

../Data/patient_genes_literature.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   FYN      977 non-null    int64 
 1   BRCA1    977 non-null    int64 
 2   FOXC1    977 non-null    int64 
 3   TBC1D1   977 non-null    int64 
 4   LAG3     977 non-null    int64 
 5   CDK6     977 non-null    int64 
 6   GATA3    977 non-null    int64 
 7   CCND1    977 non-null    int64 
 8   PRR4     977 non-null    int64 
 9   EPCAM    977 non-null    int64 
 10  CD274    977 non-null    int64 
 11  PIK3CA   977 non-null    int64 
 12  TOP2A    977 non-null    int64 
 13  DCLK1    977 non-null    int64 
 14  MYC      977 non-null    int64 
 15  LRPPRC   977 non-null    int64 
 16  BRCA2    977 non-null    int64 
 17  TP53     977 non-null    int64 
 18  MKI67    977 non-null    int64 
 19  TTN      977 non-null    int64 
 20  CTLA4    977 non-null    int64 
 21  PT

### Imports

In [11]:
from sklearn.linear_model import LogisticRegression

%run "DataHelpers.ipynb"

### Dataset split: training and test data

In [12]:
X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
print("\nApplied Smote")
X_smote, y_smote, X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote = split_data_apply_smote(df, "tnbc")

X_train.shape=(781, 30)
X_test.shape=(196, 30)
y_train.shape=(781,)
y_test.shape=(196,)

Applied Smote
X_train.shape=(1379, 30)
X_test.shape=(345, 30)
y_train.shape=(1379,)
y_test.shape=(345,)


### Logistic Regression

In [13]:
# Create model
# Bumping max_iter to a higer number than the default 100, MAY resolve the following warning
#       ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.
# TODO: Look into scaling the data

model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=100_000)


def run_model(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, test_case_ids, is_smote: bool):
    # Train the model
    model.fit(X_train, y_train)

    # Model predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

    # Save it in a dataframe, to CSV
    predictions = pd.DataFrame({
        "case_id": test_case_ids,
        "y_test": y_test,
        "y_pred": y_pred,
        "y_prob": y_prob
    })
    predictions.to_csv(f"../Data/model_output_{variant}_{GENE_FILE_VARIANT}_{'smote' if is_smote else ''}.csv", index=False)

    return y_pred, y_prob

In [14]:
y_pred, y_prod = run_model(X_train, X_test, y_train, y_test, test_case_ids, False)

print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.93


## Smote applied

In [15]:
y_pred_smote, y_prod_smote = run_model(X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote, True)

print_evaluated_model_accuracy(y_test_smote, y_pred_smote)

Accuracy: 0.94


## Model cross validation

In [18]:
def run_cross_validation(X: pd.DataFrame, y: pd.Series, y_test: pd.Series, y_pred: pd.Series, y_prob: pd.Series, is_smote: bool) -> pd.DataFrame:
    metrics = get_cross_validation_metrics(model, X, y, cv=5)
    test_metrics = get_metrics(y_test, y_pred, y_prob)
    test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
    test = pd.DataFrame([test_metrics])
    test.set_index("fold", inplace=True)

    print_validated_model_accuracy(model, metrics)

    # Prepend test_metrics to metrics dataframe, export and display
    metrics = pd.concat([test, metrics])
    metrics.to_csv(f"../Data/model_metrics_{variant}_{GENE_FILE_VARIANT}_{'smote' if is_smote else ''}.csv", index=False)
    return metrics

In [17]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metrics = run_cross_validation(X, y, y_test, y_pred, y_prod, False)
metrics

c:\Users\pauli\source\repos\School\tnbc\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\pauli\source\repos\School\tnbc\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

Model validation for LogisticRegression:
[0.9183673469387755, 0.9540816326530612, 0.958974358974359, 0.8871794871794871, 0.9179487179487179]

Mean accuracy: 0.9273



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.933673,0.652174,0.750000,0.697674,0.960794,15,168,5,8
1,0.918367,0.608696,0.666667,0.636364,0.957024,14,166,7,9
2,0.954082,0.782609,0.818182,0.800000,0.981402,18,169,4,5
3,0.958974,0.826087,0.826087,0.826087,0.940849,19,168,4,4
4,0.887179,0.434783,0.526316,0.476190,0.893327,10,163,9,13
5,0.917949,0.608696,0.666667,0.636364,0.947169,14,165,7,9


In [9]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metric_smote = run_cross_validation(X_smote, y_smote, y_test_smote, y_pred_smote, y_prod_smote, True)
metric_smote

c:\Users\pauli\source\repos\School\tnbc\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model validation for LogisticRegression:
[0.936231884057971, 0.9420289855072463, 0.9217391304347826, 0.9304347826086956, 0.936046511627907]

Mean accuracy: 0.9333



c:\Users\pauli\source\repos\School\tnbc\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.936232,0.959302,0.916667,0.937500,0.976475,165,158,15,7
1,0.936232,0.970930,0.907609,0.938202,0.967670,167,156,17,5
2,0.942029,0.970930,0.917582,0.943503,0.974862,167,158,15,5
3,0.921739,0.919075,0.924419,0.921739,0.972073,159,159,13,14
4,0.930435,0.942197,0.920904,0.931429,0.964209,163,158,14,10
5,0.936047,0.970930,0.907609,0.938202,0.965387,167,155,17,5
